Navier-Stokes on surfaces
====

In [ ]:
%run SurfNavierStokes.py

In [ ]:
from ngsolve import *
from netgen.csg import *
import netgen.gui
%gui tk

Define geometry and spaces

In [ ]:
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
Draw(mesh)

order = 5
mesh.Curve(order)

VDivSurf = HDivSurface(mesh, order = order)
VHat = HCurl(mesh, order = order, flags={"orderface": 0})
Q = SurfaceL2(mesh, order = order-1)
V = FESpace([VDivSurf,VHat,Q])

print ("ndof-Div:", VDivSurf.ndof, ", ndof-Hat:", VHat.ndof, ",ndof-L2:", Q.ndof)
((u,uhat,p),(v,vhat,q)) = V.TnT()

Navier-Stokes
\begin{align*}
\dot{u} + (u\cdot\nabla)u -\nu\Delta u + \nabla p &= f\\
\text{div}(u) &= 0
\end{align*}

HDG-Laplace

\begin{align*}
& \sum_T\int_T \nabla_{\Gamma}u\cdot\nabla_{\Gamma}v\,dx - \int_{\partial T}\nabla_{\Gamma}u_n(v-\hat{v})_{\tau}\,ds\\
& \quad - \int_{\partial T}\nabla_{\Gamma}v_n(u-\hat{u})_{\tau}\,ds+\int_{\partial T}C(u-\hat{u})_{\tau}(v-\hat{v})_{\tau}\,ds
\end{align*}

In [ ]:
a = BilinearForm(V, symmetric=True)
a += SymbolicBFI( nu*InnerProduct(gradu, gradv), BND)
a += SymbolicBFI( nu*InnerProduct( gradu.trans*n, tang(vhat-v) ), BND, element_boundary=True )
a += SymbolicBFI( nu*InnerProduct( gradv.trans*n, tang(uhat-u) ), BND, element_boundary=True )
a += SymbolicBFI( nu*4*order*order/h * InnerProduct( tang(vhat-v), tang(uhat-u) ) ,BND, element_boundary=True )
a += SymbolicBFI ( div(u)*q + div(v)*p - 1e-8*p*q, BND )
a.Assemble()

DG-Convection with upwind
\begin{align*}
\sum_T\int_T -\nabla_{\Gamma}vuu\,dx + \int_ {\partial T}u_nu^{up}v\,ds
\end{align*}
with
\begin{align*}
u^{up}=\begin{cases} u, & u_n \geq 0\\ u_nn+\hat{u}_{\tau}\tau, &\text{else}\end{cases}
\end{align*}

In [ ]:
conv = BilinearForm(V)
conv += SymbolicBFI( -InnerProduct(gradv*u, u), BND )
u_Other = (u*n)*n + tang(uhat)
conv += SymbolicBFI( u*n*IfPos(u*n, u*v, u_Other*v), BND, element_boundary = True)


In [ ]:
Draw(gfu.components[2], mesh, "pressure")
Draw(gfu.components[0], mesh, "velocity")
visoptions.scalfunction = "velocity:0"

tend = 5
t = 0
tau = 0.001

with TaskManager():
    while t < tend:
        print ("t = ", t)

        conv.Apply(gfu.vec,convvec)          
        res.data = a.mat * gfu.vec + convvec
        gfu.vec.data -= tau * inv * res    

        t += tau
        Redraw()
